based on 

https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/

https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
from custom_tensorboard import TrainValTensorBoard

Using TensorFlow backend.


In [3]:
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

In [4]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [ ]:
# load a list of files into a 3D array of [samples, timesteps, features]
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [ ]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

In [ ]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix)
    #trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix)
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [ ]:
train_x, train_y, test_x, test_y = load_dataset('../data/HAR/UCI_HAR_Dataset/')

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [ ]:
train_x.shape

(7352, 128, 9)

In [ ]:
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

In [ ]:
epochs = 20
batch_size = 50

keras imports

In [ ]:
from keras import Model
from keras.layers import Lambda, Input, Dropout, Flatten
from keras import backend as K
from keras.callbacks import TensorBoard
from time import time


In [ ]:
adam = optimizers.SGD(lr=0.01)


In [ ]:
name = 'LSTM'

model = Sequential(name =name)
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
tensorboard = TensorBoard(log_dir="logs/{}".format(name + '_' + str(time())))
model.fit(train_x, 
          train_y, epochs=epochs, 
          batch_size=batch_size, 
          validation_split=0.1,
          verbose=True,
         callbacks = [tensorboard],
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               44000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 20)                2020      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 126       
Total params: 56,246
Trainable params: 56,246
Non-trainable params: 0
_________________________________________________________________
Train on 6616 samples, validate on 736 samples
Epoch 1/20
6616/6616 [==============================] - 13s 2ms/step - loss: 1.2476 - acc: 

KeyboardInterrupt: 

### Dense network approach

some comments about this architecture:

- Note that the unstack dim is the feature dimension

In [ ]:
name = 'dense'

x = Input((n_timesteps, n_features))
unstacked = Lambda(lambda x: K.tf.unstack(x, axis=2))(x)
dense_ind = [Dense(20)(x) for x in unstacked]
merged = Lambda(lambda x: K.stack(x, axis=2))(dense_ind)
drp = Dropout(0.1)(merged)
flt1 = Flatten()(drp)
dense2 = Dense(250, activation = 'relu')(flt1)
dense2 = Dense(20, activation = 'relu')(dense2)
dense3 = Dense(n_outputs, activation='softmax')(dense2)
model = Model(x, dense3, name ='dense')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
tensorboard = TensorBoard(log_dir="logs/{}".format(name + '_' + str(time())))
model.fit(train_x, 
          train_y, epochs=epochs, 
          batch_size=batch_size, 
          validation_split=0.1,
          verbose=True,
         callbacks = [tensorboard])